# Objectif

Proposer un affichage de résultats (sous forme de pages HTML articulées entre elles) à partir du jeu de données (4 fichiers Unimarc XML d'entités LRM) et des objets Python créés dessus, à partir du code présenté dans le notebook **Manipulation des fichiers de notices Unimarc LRM**

Ce code sera amené à évoluer selon les besoins des affichages

Le coeur du dispositif est un dictionnaire d'instances de classes Item, Manif, Expression, Oeuvre et Record (pour les autres notices : personnes et sujets notamment) dont la clé est le numéro de notice, et la valeur l'instance en question.

In [1]:
from generate_records import generate_dict_entities
from generate_results import search, display_html_results, delete_html_results

### Préalable : génération du dictionnaire d'entités

In [2]:
dict_entities = generate_dict_entities("UMA_Oeuvres_Expressions.xml", "UMB_Manifestations.xml",
                                       "UMH_Items.xml", "UMA_Autres_Entites_Liees.xml")
oeuvres = [dict_entities[o] for o in dict_entities if dict_entities[o].type == "o"]

# Moteur de recherche

In [4]:
kw_search = input("Chercher dans la base : ")
results = search(kw_search, oeuvres, dict_entities, "all")
print("Nombre de résultats :", len(results))
display_html_results(results, dict_entities, kw_search)

Chercher dans la base : stoker
Nombre de résultats : 20


**[Afficher les résultats](results/short_results.html)**

### Supprimer les pages de résultats

Lancer la cellule ci-dessous

In [6]:
delete_html_results()